In [55]:
import requests as req
import os
import json
from pprint import pprint

import numpy as np
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000


In [163]:
#### Download Google Spreadsheets ####

# Make a temporary directory, and move into it
!mkdir temp
os.chdir("temp")
dest = os.getcwd()

# July Data Sheet
!curl "https://docs.google.com/spreadsheets/d/1viPOGYIk6RGu7YMoM3BHNVbkWaCZ0JFBOMSNncWvHYk/export?format=tsv" > july_data_upload.tsv
july_data_upload = pd.read_csv(dest+"/july_data_upload.tsv", sep="\t", index_col=[0])

# # Metadata to Upload
# !curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
# current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
# current_mdata = current_mdata.transpose()

!curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
current_mdata = current_mdata.transpose()

# Delete temporary files
os.chdir("..")
!rm -r temp


#### Merge info from the Tracking sheet and Metadata sheet

# Add RW API key to metadata_to_upload
def fetch_info(row, match_col, dest_df, target_column):
    try:
        info = dest_df.loc[row[match_col], target_column]
        return(info)
    except:
        return(None)

old_id_col = "VIZZ Unique ID - RW API (bulk upload)"
new_id_col = "API-ID (PERFECT DATASET)"
dl_from_src_col = "Download from Source"
dl_from_s3_col = "Download Data (S3)"

match_col = "Unique ID"

current_mdata[old_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,old_id_col), axis=1)
current_mdata[new_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,new_id_col), axis=1)

current_mdata[dl_from_src_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_src_col), axis=1)
current_mdata[dl_from_s3_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_s3_col), axis=1)

# Keep only those datasets with rw_ids already
valid_old_ids = pd.notnull(current_mdata[old_id_col])
valid_new_ids = pd.notnull(current_mdata[new_id_col])

def choose_new_id(df, valid_old_ids,old_id_col, valid_new_ids,new_id_col):
    assert(len(valid_old_ids)==len(valid_new_ids))
    final_ids = []
    for i in range(0, len(valid_new_ids)):
        if(valid_new_ids[i]):
            final_ids.append(df.iloc[i][new_id_col])
        elif(valid_old_ids[i]):
            final_ids.append(df.iloc[i][old_id_col])
        else:
            final_ids.append(None)
    return(final_ids)

current_mdata["final_ids"] = choose_new_id(current_mdata, valid_old_ids,old_id_col, valid_new_ids,new_id_col)
keep_matched_ids = pd.notnull(current_mdata["final_ids"])

current_mdata = current_mdata.loc[keep_matched_ids]
current_mdata.set_index("final_ids", inplace=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 93191    0 93191    0     0   122k      0 --:--:-- --:--:-- --:--:--  122k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  569k    0  569k    0     0  1660k      0 --:--:-- --:--:-- --:--:-- 1661k


In [131]:
list1 = [None, "a", None, "b"]
list2 = ["c", None, None, "d"]
df = pd.DataFrame({"l1":list1, "l2":list2})
choose_new_id(df, list1,"l1", list2,"l2")

['c', 'a', None, 'd']

In [164]:
print(current_mdata.shape)
print(july_data_upload.shape)

(109, 42)
(290, 35)


In [165]:
current_mdata.head(1)

,Public Title,Technical Title,Subtitle,Unique ID,Source Organizations,Learn More Link,Function,Description,Cautions,Geographic Coverage,Data Type,Spatial Resolution,Date of Content,Frequency of Updates,Summary of Licence,Link to License,Citation,Published Language,Published Title (if not English),Download,Download from Source,Uploaded To,Layer Name 1,Layer Definition 1,Layer Name 2,Layer Definition 2,Layer Name 3,Layer Definition 3,Layer Name 4,Layer Definition 4,Original Data Name 1,Original Data Link 1,Original Data Name 2,Original Data Link 2,Original Data Name 3,Original Data Link 3,Original Data Name 4,Original Data Link 4,nan,VIZZ Unique ID - RW API (bulk upload),API-ID (PERFECT DATASET),Download Data (S3)
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93ee443e-cb39-424a-9aa4-1d16af813418,Endangered Species Sites,NaN,AZE,bio.001,Alliance for Zero Extinction (AZE),http://www.biodiversitya-z.org/content/allianc...,Global list of sites containing 95% or more of...,Created by the Alliance for Zero Extinction (A...,None listed.,Global,Vector,NaN,2010,5 years,The AZE data and any derivatives may be used f...,https://www.arcgis.com/home/item.html?id=4ecca...,"Alliance for Zero Extinction. 2010. ""2010 AZE ...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93ee443e-cb39-424a-9aa4-1d16af813418,93ee443e-cb39-424a-9aa4-1d16af813418,NaN


In [166]:
july_data_upload.head(1)

,VIZZ Unique ID - RW API (bulk upload),Slug - RW API (sanity check),API-ID (PERFECT DATASET),Public Title,Alternative Public Title,Technical Title,Subtitle,Theme_1,Theme_2,Theme_3,Planet Pulse,Water & Conflict,Problem Solving,Process these first,Multiple Layers needed to surface at same time?,Metadata Completed,Metadata on Backoffice,Distribution Restriction,Tags,Format,"Endpoint URL (Carto, GEE)",Download from Source,Download Data (S3),On WRI Platform,Dataset Processed for Upload,Data Upload Responsibility,Uploaded to S3,Server Location,Missing ISO Code,Dataset on Backoffice,Columns defined on Backoffice,Layer Definition,Layer Name/Description finalized,Widgets,Published on RW
WRI Unique ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bio.001,93ee443e-cb39-424a-9aa4-1d16af813418,Alliance-for-Zero-Extinction-sites,93ee443e-cb39-424a-9aa4-1d16af813418,Endangered Species Sites,NaN,Alliance for Zero Extinction (AZE) Endangered ...,AZE,Biodiversity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN,GFW,NaN,Vizz,NaN,Carto,NaN,X,NaN,X,X,NaN,X


In [167]:
current_mdata.columns

Index([                         'Public Title',
                             'Technical Title',
                                    'Subtitle',
                                   'Unique ID',
                        'Source Organizations',
                             'Learn More Link',
                                    'Function',
                                 'Description',
                                    'Cautions',
                         'Geographic Coverage',
                                   'Data Type',
                          'Spatial Resolution',
                             'Date of Content',
                        'Frequency of Updates',
                          'Summary of Licence',
                             'Link to License',
                                    'Citation',
                          'Published Language',
            'Published Title (if not English)',
                                    'Download',
                        'Download from S

In [168]:
## FOR EACH DATASET IN BACKOFFICE THAT HAS METADATA, UPLOAD IT
from configparser import ConfigParser
config = ConfigParser()
config.read("../.env")
api_token = config.get("configuration", "api_token")

auth_token = api_token # <Insert Auth Token Here>
def clean_nulls(val):
    try:
        if np.isnan(val):
            return(None)
        else:
            return(val)
    except:
        return(val)

#test = ["3624554e-b240-4edb-9110-1f010642c3f3"]



processed1 = []


### THIS ADDS ALL DATASETS FOR WHICH WE HAVE METADATA in METADATA FOR UPLOAD ###

for rw_id in current_mdata.index:
#for rw_id in test:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    print(url)
    # Everything from current_mdata
    metadata = current_mdata.loc[rw_id]
    if len(metadata.shape) > 1:
        print(metadata)
    
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": clean_nulls(metadata["Public Title"]),
        "description": clean_nulls(metadata["Description"]),
        "source": clean_nulls(metadata["Subtitle"]),
        "functions": clean_nulls(metadata["Function"]),
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            "wri_rw_id": clean_nulls(metadata["Unique ID"]),
            "data_type": clean_nulls(metadata["Data Type"]),

            "name": clean_nulls(metadata["Public Title"]),
            "source_organization": clean_nulls(metadata["Source Organizations"]),
            "technical_title":clean_nulls(metadata["Technical Title"]),

            "function": clean_nulls(metadata["Function"]),
            "cautions": clean_nulls(metadata["Cautions"]),
            
            "citation": clean_nulls(metadata["Citation"]),
            "summary_of_license": clean_nulls(metadata["Summary of Licence"]),
            "link_to_license": clean_nulls(metadata["Link to License"]),
            
            "geographic_coverage": clean_nulls(metadata["Geographic Coverage"]),
            "spatial_resolution": clean_nulls(metadata["Spatial Resolution"]),
            "date_of_content": clean_nulls(metadata["Date of Content"]),
            "frequency_of_updates": clean_nulls(metadata["Frequency of Updates"]),
            
            "learn_more_link": clean_nulls(metadata["Learn More Link"]),
            "data_download_link": clean_nulls(metadata["Download Data (S3)"]),
            "data_download_original_link":clean_nulls(metadata["Download from Source"])
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }

    try:
        processed1.append(rw_id)
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        print(res)
        print(res.text)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            print(res)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])

https://api.resourcewatch.org/v1/dataset/93ee443e-cb39-424a-9aa4-1d16af813418/metadata
<Response [400]>
{"errors":[{"status":400,"detail":"Metadata of resource dataset: 93ee443e-cb39-424a-9aa4-1d16af813418, application: rw and language: en already exists"}]}
<Response [200]>
https://api.resourcewatch.org/v1/dataset/4458eb12-8572-45d1-bf07-d5a3ee097021/metadata
<Response [400]>
{"errors":[{"status":400,"detail":"Metadata of resource dataset: 4458eb12-8572-45d1-bf07-d5a3ee097021, application: rw and language: en already exists"}]}
<Response [200]>
https://api.resourcewatch.org/v1/dataset/16df8ada-87cc-4907-adce-a98bc4e91856/metadata
<Response [400]>
{"errors":[{"status":400,"detail":"Metadata of resource dataset: 16df8ada-87cc-4907-adce-a98bc4e91856, application: rw and language: en already exists"}]}
<Response [200]>
https://api.resourcewatch.org/v1/dataset/3624554e-b240-4edb-9110-1f010642c3f3/metadata
<Response [400]>
{"errors":[{"status":400,"detail":"Metadata of resource dataset: 362

In [169]:
#### Merge subtitles with Tracking sheet

#### Find out which datasets have a RW Unique ID, but aren't in Metadata for Upload
# Keep only those datasets with rw_ids already
tracking_valid_old_ids = pd.notnull(july_data_upload[old_id_col])
tracking_valid_new_ids = pd.notnull(july_data_upload[new_id_col])

july_data_upload["final_ids"] = choose_new_id(july_data_upload, tracking_valid_old_ids,old_id_col, tracking_valid_new_ids,new_id_col)

missed_ids = [rw_id for rw_id in july_data_upload["final_ids"].values if ((rw_id not in current_mdata.index) and (rw_id != None))]

missed_data = july_data_upload.reset_index().set_index("final_ids")
missed_data = missed_data.loc[missed_ids]
missed_data


### THIS ADDS ALL DATASETS FOR WHICH WE HAVE ENTRIES IN TRACKING SHEET and NOTHING IN METADATA FOR UPLOAD###
print("True if below print empty list []")
print([ind for ind in missed_data.index if ind in current_mdata.index])

processed2 = []

for rw_id in missed_data.index:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    # Everything from current_mdata
    metadata = missed_data.loc[rw_id]
    print(metadata["WRI Unique ID"])
    print(metadata["Public Title"])
    print(url)
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": clean_nulls(metadata["Public Title"]),
        "source": clean_nulls(metadata["Subtitle"]),
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            "wri_rw_id": clean_nulls(metadata["WRI Unique ID"]),

            "name": clean_nulls(metadata["Public Title"]),
            "technical_title":clean_nulls(metadata["Technical Title"]),

            "data_download_link": clean_nulls(metadata["Download Data (S3)"]), 
            "data_download_original_link": clean_nulls(metadata["Download from Source"])
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }
    #print(row_payload)

    try:
        processed2.append(rw_id)
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])



True if below print empty list []
[]
bio.022.1
Observations of Coral Disease
https://api.resourcewatch.org/v1/dataset/20c70a51-4ddf-4f6c-ad2c-1a6729b95fa4/metadata
bio.031
Social and Economic Dependence on Coral Reefs
https://api.resourcewatch.org/v1/dataset/894f43a8-ce8e-43a5-a4c7-fa80faa43d63/metadata
bio.033
Cold Water Corals
https://api.resourcewatch.org/v1/dataset/6b8442f5-4766-4444-94b4-d6676277fd80/metadata
bio.035
Coral Bleaching Frequency Prediction
https://api.resourcewatch.org/v1/dataset/1ef55baf-bbbe-480d-85e9-7132c742f196/metadata
cit.032
Urban Population Projections
https://api.resourcewatch.org/v1/dataset/7d9c0d09-e833-4a74-811b-0af78da9c731/metadata
cli.003
Sea Surface Temperature Anomalies
https://api.resourcewatch.org/v1/dataset/c8040a7a-a40f-48bd-b003-625c33beff5e/metadata
cli.013
CH4 Concentrations
https://api.resourcewatch.org/v1/dataset/c9c9cb2f-9655-4f40-8736-9b407ee43514/metadata
cli.021
Average Snow Cover
https://api.resourcewatch.org/v1/dataset/ea67f436-473c-4

In [138]:
"8f14a33e-5a61-47af-b26e-c1fc036932a5" in processed

True

[]

In [171]:
missed_data.to_csv("/Users/nathansuberi/Desktop/datasets_on_july_sheet_with_rw_id_no_metadata.csv")
missed_data

,WRI Unique ID,VIZZ Unique ID - RW API (bulk upload),Slug - RW API (sanity check),API-ID (PERFECT DATASET),Public Title,Alternative Public Title,Technical Title,Subtitle,Theme_1,Theme_2,Theme_3,Planet Pulse,Water & Conflict,Problem Solving,Process these first,Multiple Layers needed to surface at same time?,Metadata Completed,Metadata on Backoffice,Distribution Restriction,Tags,Format,"Endpoint URL (Carto, GEE)",Download from Source,Download Data (S3),On WRI Platform,Dataset Processed for Upload,Data Upload Responsibility,Uploaded to S3,Server Location,Missing ISO Code,Dataset on Backoffice,Columns defined on Backoffice,Layer Definition,Layer Name/Description finalized,Widgets,Published on RW
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20c70a51-4ddf-4f6c-ad2c-1a6729b95fa4,bio.022.1,NaN,NaN,20c70a51-4ddf-4f6c-ad2c-1a6729b95fa4,Observations of Coral Disease,NaN,Global Observations of Coral Disease,ReefBase,Biodiversity,NaN,NaN,NaN,NaN,Peter to download/upload,NaN,NaN,X,NaN,X,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://www.reefbase.org/gis_maps/datasets.aspx,NaN,NaN,NaN,Peter,X,Carto,NaN,X,NaN,X,X,NaN,X
894f43a8-ce8e-43a5-a4c7-fa80faa43d63,bio.031,894f43a8-ce8e-43a5-a4c7-fa80faa43d63,NaN,894f43a8-ce8e-43a5-a4c7-fa80faa43d63,Social and Economic Dependence on Coral Reefs,NaN,Social and Economic Dependence on Coral Reefs,WRI,Biodiversity,Commerce,Society,NaN,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://datasets.wri.org/dataset/reefs-at-risk-...,NaN,NaN,NaN,Liz,NaN,Carto,NaN,X,NaN,X,X,NaN,X
6b8442f5-4766-4444-94b4-d6676277fd80,bio.033,6b8442f5-4766-4444-94b4-d6676277fd80,NaN,6b8442f5-4766-4444-94b4-d6676277fd80,Cold Water Corals,NaN,Global DIstribution of Cold Water Corals,UNEP-WCMC,Biodiversity,NaN,NaN,NaN,NaN,Max/Laura write metadata,NaN,NaN,ready,NaN,X,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://data.unep-wcmc.org/datasets/3,https://wri-public-data.s3.amazonaws.com/resou...,NaN,NaN,Elise,X,Carto,NaN,X,NaN,X,X,NaN,X
1ef55baf-bbbe-480d-85e9-7132c742f196,bio.035,1ef55baf-bbbe-480d-85e9-7132c742f196,NaN,NaN,Coral Bleaching Frequency Prediction,NaN,Frequency of Future Coral Reef Bleaching Event...,WRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7d9c0d09-e833-4a74-811b-0af78da9c731,cit.032,7d9c0d09-e833-4a74-811b-0af78da9c731,NaN,NaN,Urban Population Projections,NaN,Urban Population Percentage,No Metadata Yet,Cities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ready,NaN,X,NaN,CSV,https://resourcewatch.carto.com/u/wri-rw/datas...,https://esa.un.org/unpd/wup/CD-ROM/,https://wri-projects.s3.amazonaws.com/resource...,NaN,NaN,Emily,X,Carto,NaN,X,NaN,X,NaN,X,X
c8040a7a-a40f-48bd-b003-625c33beff5e,cli.003,c8040a7a-a40f-48bd-b003-625c33beff5e,Sea-Surface-Temperature-Anomalies-NOAA,NaN,Sea Surface Temperature Anomalies,NaN,Twice-weekly Sea Surface Temperature Anomalies,No Metadata Yet,Climate,NaN,NaN,RT,NaN,Vizz rasters - who makes layer?,NaN,NaN,ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,?,NaN,NaN,X
c9c9cb2f-9655-4f40-8736-9b407ee43514,cli.013,c9c9cb2f-9655-4f40-8736-9b407ee43514,Current-Methane-Concentration-CH4,NaN,CH4 Concentrations,NaN,"Current Methane Concentration, CH4",No Metadata Yet,Climate,NaN,NaN,RT,NaN,Liz to check,NaN,NaN,Liz,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,X,NaN,NaN,NaN
ea67f436-473c-4977-bdf2-8aa0dabbaa6f,cli.021,ea67f436-473c-4977-bdf2-8aa0dabbaa6f,NaN,NaN,Average Snow Cover,NaN,Northern Hemisphere Average Snow Cover by Mont...,No Metadata Yet,Climate,NaN,NaN,RT,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Raster,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN
d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,cli.025,d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,Sea-Level-Trends,NaN,Relative Sea Level Trends,NaN,Global Linear Relative Mean Sea Level (MSL) tr...,No Metadata Yet,Climate,Cities,NaN,explore,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,NaN,NaN,https

In [172]:
#### Inspect metadata on backoffice



# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "upload_name":atts["name"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata_keys":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }    
    
# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
def check_public_title(metadata):
    if len(metadata) > 0:
        mdata = metadata[0]
        if "attributes" in mdata:
            if "info" in mdata["attributes"]:
                if "name" in mdata["attributes"]["info"]:
                    return(mdata["attributes"]["info"]["name"])
        return(None)

current_datasets_on_api["public_title"] = current_datasets_on_api.apply(lambda row: check_public_title(row["metadata"]), axis=1)
current_datasets_on_api["without_public_title"] = current_datasets_on_api.apply(lambda row: check_public_title(row["metadata"]), axis=1)




current_datasets_on_api.set_index("rw_id", inplace=True)
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

In [66]:
current_datasets_on_api.loc["0087944f-871c-44bc-b4d9-cd5acfc27023", "layers"]

[{'attributes': {'application': ['rw'],
   'applicationConfig': {},
   'dataset': '0087944f-871c-44bc-b4d9-cd5acfc27023',
   'default': True,
   'description': '',
   'env': 'production',
   'interactionConfig': {},
   'iso': [],
   'layerConfig': {'body': {'attribution': '2016 wri/google/Hansen',
     'errorTileUrl': 'https://storage.googleapis.com/global-surface-water/downloads_ancillary/blank.png',
     'format': 'image/png',
     'maxZoom': 13,
     'transparent': True},
    'service': 'leaflet',
    'type': 'tileLayer',
    'url': 'http://staging-api.globalforestwatch.org/v1/true-color-tiles/loss/{z}/{x}/{y}'},
   'legendConfig': {'items': [{'color': '#dc1e8c',
      'name': 'Loss (2001-2016)'}],
    'type': 'basic'},
   'name': 'Forest loss',
   'provider': 'leaflet',
   'slug': 'Forest-loss',
   'staticImageConfig': {},
   'updatedAt': '2017-10-06T10:07:30.025Z',
   'userId': '57a0aa1071e394dd32ffe137'},
  'id': '6d29ffbc-11d3-4cd7-b2d5-25f0ff7ffb77',
  'type': 'layer'},
 {'attr

In [174]:
### THIS COVERS ALL DATASETS WHICH ARE ON THE BACKOFFICE but HAVE NO RW_ID IN TRACKING SHEET ###

investigate_mdata = current_datasets_on_api[["upload_name", "public_title", "metadata"]]

missed_ids = [rw_id for rw_id in investigate_mdata.index if ((rw_id not in processed1) & (rw_id not in processed2))]

investigate_mdata = investigate_mdata.loc[missed_ids]

print("8f14a33e-5a61-47af-b26e-c1fc036932a5" in missed_ids)
print("8f14a33e-5a61-47af-b26e-c1fc036932a5" in processed)
print("8f14a33e-5a61-47af-b26e-c1fc036932a5" in investigate_mdata.index)

investigate_mdata

False
True
False


,upload_name,public_title,metadata
Dataset,,,
415dd499-8385-4c23-bd18-8ac49803cf9e,(marked after launch) foo.002 GLDAS Land Water...,Land Water Content,"[{'id': '59a426727b6c000012baa6f9', 'type': 'm..."
88648f97-35a5-4384-b141-ac00b4f66209,Dash_0057 Trends in Protected Area Coverage of...,None,[]
32324f22-e7f6-4e0d-bc38-d82c6484f1be,(delete?) For.008 Country Annual Tree Cover Loss,T1_test,"[{'id': '59dccf37c9f39800126e32bc', 'type': 'm..."
d7a6f475-7e4a-4d02-b412-acbdb0f1c005,TEST PREP 001 Prep Population,None,[]
32213dbc-2ec1-4579-a9c8-21afa2793bb6,(delete?) Drought Monitor,None,[]
63e29bc8-c341-44a4-bc0e-1d3dc1714c13,(?) Floods News Reports,None,[]
bf27ddaa-1117-49f7-b763-7a166a0b27ce,(?) Open Air Quality: BC.,None,[]
8d84f6c6-8e7d-410e-a408-cbfb2555b35d,(delete?) Conflict and Protest Data for Africa...,Conflict and Protest Events in African States,"[{'id': '59664b3f0d9ed1000bc29a95', 'type': 'm..."
1170030b-9725-4bfe-8fb4-1b0eb81020d2,(delete?) Most polluting countries in 2011,None,[]


In [146]:
current_datasets_on_api

,upload_name,table_name,provider,date_updated,num_metadata_keys,metadata,num_layers,layers,num_widgets,widgets,num_tags,tags,public_title
Dataset,,,,,,,,,,,,,
0448c79d-0ee0-42ff-9331-aeee70cef301,for.008 Annual Tree Cover Loss,UMD/hansen/global_forest_change_2015_v1_3,gee,2017-10-10T19:39:38.112Z,1,"[{'id': '59dd226526cb7a0013147aab', 'type': 'm...",0,[],0,[],0,[],Annual Tree Cover Loss
e2869a81-6b83-49b2-a50d-428c54b72593,for.007 Annual Tree Cover Gain,UMD/hansen/global_forest_change_2015_v1_3,gee,2017-10-10T19:38:32.119Z,1,"[{'id': '59dd22658e6ae800119b3e44', 'type': 'm...",0,[],0,[],0,[],Annual Tree Cover Gain
d7a6f475-7e4a-4d02-b412-acbdb0f1c005,TEST PREP 001 Prep Population,prep_001_pop,cartodb,2017-10-10T19:07:55.930Z,0,[],1,[{'id': '89f19b69-d03f-4030-9792-89945fd0b608'...,1,[{'id': '867fc03b-2651-490c-8edb-b4f86a3a2bf4'...,0,[],None
c36c3108-2581-4b68-852a-c929fc758001,dis_007 Landslide Susceptibility Map,users/resourcewatch/dis_007_landslide_suscepti...,gee,2017-10-10T17:40:24.563Z,0,[],1,[{'id': '42caee24-4225-4c0c-8c1b-4da550ddd120'...,0,[],0,[],None
63a7a997-695d-4629-b6e9-9b169f5c69bf,wat.030 World Hydro Reference,,wms,2017-10-10T17:27:14.913Z,1,"[{'id': '59dd23f426cb7a0013147ab8', 'type': 'm...",1,[{'id': '77ba0a3b-3940-4525-90ad-f1c729bc8673'...,0,[],1,"[{'type': 'vocabulary', 'attributes': {'resour...",World Hydro Reference
32213dbc-2ec1-4579-a9c8-21afa2793bb6,(delete?) Drought Monitor,spei,cartodb,2017-10-10T15:58:47.786Z,0,[],1,[{'id': '6f1f8b5d-a6ba-436f-a7bf-b6812d0f3a20'...,0,[],1,"[{'type': 'vocabulary', 'attributes': {'resour...",None
4c9190e9-205b-4ff8-83dd-ee08cd63a04c,soc_005 Political Rights and Civil Liberties I...,fiw2017,cartodb,2017-10-10T14:27:00.869Z,1,"[{'id': '59a427ac7b6c000012baa70b', 'type': 'm...",1,[{'id': '7c334205-2a0c-4d24-8454-244c2990dc30'...,1,[{'id': '5d4dc196-709d-4201-8877-86b8598ea1e3'...,1,"[{'type': 'vocabulary', 'attributes': {'resour...",Political Rights and Civil Liberties Index
6f19982e-8c27-45ae-936d-2c9d5b2bd6a6,dis.012 Potential Landslide Areas,global_landslide_nowcast_3hr,cartodb,2017-10-10T14:02:50.106Z,1,"[{'id': '59dce62f991d67001321650d', 'type': 'm...",1,[{'id': '50ef9f99-ddca-46b9-bb69-690f0b2ced88'...,1,[{'id': '9a6e1e42-6436-4e59-96cf-f5d1391bd8b4'...,1,"[{'type': 'vocabulary', 'attributes': {'resour...",Landslide Hazards
63e29bc8-c341-44a4-bc0e-1d3dc1714c13,(?) Floods News Reports,flood_observatory,cartodb,2017-10-10T13:58:17.202Z,0,[],1,[{'id': 'd0e35d92-3669-4bc2-b5d6-58b456fbf847'...,1,[{'id': '94b43a46-5594-46fe-8484-46e0dd73197b'...,1,"[{'type': 'vocabulary', 'attributes': {'resour...",None


In [133]:
"10337db6-8321-445e-a60b-28fc1e114f29" in processed

False

These are datasets for which:
1) There 

soc.003 Distribution of Infant Mortality
soc.016 Conflict and Protest Events in African...
dis_007 Landslide Susceptibility Map
bio.035 Coral Bleaching Frequency Prediction
dis.001 Earthquakes Over the Past 30 days
Foo_046a Food Footprint in Protein
wat.033 Agriculture Water Demand and Depletion
soc.062 Internal Displacement
soc.061 Rural Poverty
soc.042 Percentage of Urban Population with Ac
soc.020 GINI Index
soc.008 Gross Domestic Product Per Capita (PPP
soc.006 Multidimensional Poverty Index
soc.004 Human Development Index
soc.002 Gender Development Index
foo.002 GLDAS Land Water Content from NOAH Lan..
com.028 Effect of Agricultural Policies on Com...
cit.029 Municipal Waste

test upload cit.029:
    broken: 8f14a33e-5a61-47af-b26e-c1fc036932a5
    working: 00abb46f-34e2-4bf7-be30-1fb0b1de022f

In [158]:

        
url1="https://api.resourcewatch.org/v1/dataset/8f14a33e-5a61-47af-b26e-c1fc036932a5/metadata"    
url2="https://api.resourcewatch.org/v1/dataset/10337db6-8321-445e-a60b-28fc1e114f29/metadata"

res1a = req.request("POST", url1, data=json.dumps(row_payload), headers = headers)
if("already exists" in res1a.text):
    res1b = req.request("PATCH", url1, data=json.dumps(row_payload), headers = headers)
        
res2a = req.request("POST", url2, data=json.dumps(row_payload), headers = headers)
if("already exists" in res2a.text):
    res2b = req.request("PATCH", url1, data=json.dumps(row_payload), headers = headers)



In [162]:
res1b.text

'{"data":[{"id":"59d538ff66b9630011465ece","type":"metadata","attributes":{"dataset":"8f14a33e-5a61-47af-b26e-c1fc036932a5","application":"rw","resource":{"type":"dataset","id":"8f14a33e-5a61-47af-b26e-c1fc036932a5"},"language":"en","name":"Annual Precipitation","description":"The Municipal Waste Generated per Capita data represent trends in amounts of municipal waste (including household) produced, and the treatment and disposal method used. They are provided by OECD member countries\' authorities through the questionnaire on the state of the environment (OECD/Eurostat). They were updated or revised on the basis of data from other national and international sources available to the OECD Secretariat, and on the basis of comments received from national delegates. Selected updates were also done in the context of the OECD Environmental Performance Reviews. The data are harmonized through the work of the OECD Working Party on Environmental Information (WPEI) and benefit from continued dat